In [298]:
# Understanding supervised learning
# Supervised - Data points have known outcomes
# Unsupervised - Data points have unknown outcomes

# Regression - Output is numerical (Continous)
# Classification - Output is categorical

# Data with answers + Model -> Fit (Gives an Equation)
# Data without answers + Fit_Model -> Predict -> Predicted Answers

# Dataframes and data structures for Data Science
import pandas as pd

# Mathematical computation /if required
import numpy as np

In [299]:
# Data prerocessing with Sklearn
# Data scalars - These are your buddies for algorithms that use distance based calculations, if so then the scaling of
# diferent kinds of attributes needs to be done. Different kinds of scaling options are avalibale.

# 1. Standard Scaling - Scales from the mean in terms of UNIT VARIANCE
# 2. Minimun - Maximum Scaling - Scale to fixed range data e.g. - (5-10)
# 3. Maximum Absolute Value Scaling - Scale from 0 to maximum number to bring it between 0s and 1s, we should be
#    careful with negative values!

# STANDARD SCALAR
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler

In [301]:
# Create the scalar object

stdSc = StandardScaler()
minMax = MinMaxScaler()
absSc = MaxAbsScaler()

In [407]:
# KNN Classification
# Stores all available cases -> new data point -> case based on a similarity

# k - Number of nearest neighbours

# KNN - Recommender system
#     - Document Filtering - Finding the concept

# Knn working - Similarity based algorithm
#             - Assign a point, ask which category it belongs to class b?
#             - Increasing k, the radius increases
#             - How to choose k? Trial and Error if 1000 then 1-19 is ideal k value
#             - L2 : Eucledian distance (Direct Distance)
#             - L1: Manhattan distance (Roads Distance) Adding absolute deltas

# Lazy Learner - Tries to memorize the data, only during prediction, tries to plot the point 

# Let's use the Iris Data Set as an example

# Telecom Customer Churn Prediction Data
# Import the dataset 
df = pd.read_csv('Data/Orange_Telecom_Churn_Data.csv')

# Unnecessary categorical data
df = df.drop(['phone_number','area_code','state'], 1)

# Creating dummy columns
df = dummyAndBindBinary(df,'voice_mail_plan')
df = dummyAndBindBinary(df,'intl_plan')

df = attachTargetAtTheEnd(df, 'churned')
msc = MinMaxScaler()
df = pd.DataFrame(msc.fit_transform(df),  # this is an np.array, not a dataframe.
                    columns=df.columns)

df.head()

,account_length,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,voice_mail_plan_no,voice_mail_plan_yes,intl_plan_no,intl_plan_yes,churned
0,0.524793,0.480769,0.754196,0.666667,0.754183,0.542755,0.582353,0.542866,0.619494,0.520000,0.619584,0.500,0.15,0.500000,0.111111,0.0,1.0,1.0,0.0,0.0
1,0.438017,0.500000,0.459744,0.745455,0.459672,0.537531,0.605882,0.537690,0.644051,0.588571,0.644344,0.685,0.15,0.685185,0.111111,0.0,1.0,1.0,0.0,0.0
2,0.561983,0.000000,0.692461,0.690909,0.692436,0.333242,0.647059,0.333225,0.411646,0.594286,0.411930,0.610,0.25,0.609259,0.000000,1.0,0.0,1.0,0.0,0.0
3,0.342975,0.000000,0.851778,0.430303,0.851740,0.170195,0.517647,0.170171,0.498481,0.508571,0.498593,0.330,0.35,0.329630,0.222222,1.0,0.0,0.0,1.0,0.0
4,0.305785,0.000000,0.474253,0.684848,0.474230,0.407754,0.717647,0.407959,0.473165,0.691429,0.473270,0.505,0.15,0.505556,0.333333,1.0,0.0,0.0,1.0,0.0


In [397]:
# Defining an easy function for dummyfying and binding

def dummyAndBindBinary(df, attribute):
    dfvm = pd.get_dummies(df[attribute])
    dfvm.columns = [attribute + '_no',attribute + '_yes']
    df = pd.concat([df, dfvm], axis = 1)
    df = df.drop([attribute], axis = 1)
    return df

def dummyAndBind(df, attribute):
    dfvm = pd.get_dummies(df[attribute])
    df = pd.concat([df, dfvm], axis = 1)
    df = df.drop([attribute], axis = 1)
    return df

def attachTargetAtTheEnd(df, target):
    tarCol = df[target]
    df = df.drop([target], axis = 1)
    df = pd.concat([df, tarCol], axis = 1)
    return df
    

In [525]:
# Split into test and train data
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3)

# Use the Knn Algorithm
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=9)
KNN = KNN.fit(X=train.iloc[:,:-1],y=train.iloc[:,-1])

predicted = pd.Series(KNN.predict(X=test.iloc[:,:-1]))

# Because of random selection of indexes, jumbled indexes are present
predicted.index = test.index
test['churned_knn'] = predicted

# Accuracy calculation
from sklearn import metrics
print("Model Accuracy on Test:",(metrics.accuracy_score(test['churned'], test['churned_knn'])*100),"%")

Model Accuracy on Test: 91.73333333333333 %


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [538]:
# Performing Weighted KNN
# Weighted KNN means you're altering the influence of a data point based on some function, here I'm using distance as that functio
# Weighted KNN 'distance' means that the value of weight is inversely proportional to the distance between the two points

# Split into test and train data
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3)

# Use the Knn Algorithm
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=9,weights='distance')
KNN = KNN.fit(X=train.iloc[:,:-1],y=train.iloc[:,-1])

predicted = pd.Series(KNN.predict(X=test.iloc[:,:-1]))

# Because of random selection of indexes, jumbled indexes are present
predicted.index = test.index
test['churned_knn'] = predicted

# Accuracy calculation
from sklearn import metrics
print("Model Accuracy on Test:",(metrics.accuracy_score(test['churned'], test['churned_knn'])*100),"%")

Model Accuracy on Test: 92.06666666666666 %


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [553]:
# Performing Weighted KNN with Manhattan Distance
# Weighted KNN means you're altering the influence of a data point based on some function, here I'm using distance as that functio
# Weighted KNN 'distance' means that the value of weight is inversely proportional to the distance between the two points

# Split into test and train data
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3)

# Use the Knn Algorithm
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=3,weights='distance',p=1)
KNN = KNN.fit(X=train.iloc[:,:-1],y=train.iloc[:,-1])

predicted = pd.Series(KNN.predict(X=test.iloc[:,:-1]))

# Because of random selection of indexes, jumbled indexes are present
predicted.index = test.index
test['churned_knn'] = predicted

# Accuracy calculation
from sklearn import metrics
print("Model Accuracy on Test:",(metrics.accuracy_score(test['churned'], test['churned_knn'])*100),"%")

Model Accuracy on Test: 91.4 %


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
